In [153]:
import boto3
import sagemaker
import pandas as pd
import seaborn as sns
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

In [154]:
s3_client = boto3.client('s3', region_name='us-east-1')
bucket_name = 'team1ads508finalproject'

In [155]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

s3_public = "s3://team1ads508finalproject"

s3_private = "s3://{}/ADS508_project".format(bucket)

!aws s3 cp --recursive $s3_public/ $s3_private/

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
copy: s3://team1ads508finalproject/application_train.csv to s3://sagemaker-us-east-1-654654578488/ADS508_project/application_train.csv
copy: s3://team1ads508finalproject/bureau.csv to s3://sagemaker-us-east-1-654654578488/ADS508_project/bureau.csv
copy: s3://team1ads508finalproject/previous_application.csv to s3://sagemaker-us-east-1-654654578488/ADS508_project/previous_application.csv


In [156]:
print(s3_private)

s3://sagemaker-us-east-1-654654578488/ADS508_project


In [157]:
%store

Stored variables and their in-db values:


In [158]:
database_name = "dsoaws"

In [159]:
#Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [160]:
conn = connect(region_name='us-east-1', s3_staging_dir=s3_staging_dir)

In [162]:
# Define the table schemas for each CSV file
schemas = {
    'bureau': """
        CREATE EXTERNAL TABLE IF NOT EXISTS bureau (
            SK_ID_CURR INT,
            SK_ID_BUREAU INT,
            CREDIT_ACTIVE STRING,
            CREDIT_CURRENCY STRING,
            DAYS_CREDIT INT,
            CREDIT_DAY_OVERDUE INT,
            DAYS_CREDIT_ENDDATE INT,
            DAYS_ENDDATE_FACT INT,
            AMT_CREDIT_MAX_OVERDUE FLOAT,
            CNT_CREDIT_PROLONG INT,
            AMT_CREDIT_SUM FLOAT,
            AMT_CREDIT_SUM_DEBT FLOAT,
            AMT_CREDIT_SUM_LIMIT FLOAT,
            AMT_CREDIT_SUM_OVERDUE FLOAT,
            CREDIT_TYPE STRING,
            DAYS_CREDIT_UPDATE INT,
            AMT_ANNUITY FLOAT
        )
        ROW FORMAT DELIMITED
        FIELDS TERMINATED BY ','
        LOCATION 's3://team1ads508finalproject/'
    """,
    'previous_application': """
        CREATE EXTERNAL TABLE IF NOT EXISTS previous_application (
            SK_ID_PREV INT,
            SK_ID_CURR INT,
            NAME_CONTRACT_TYPE STRING,
            AMT_ANNUITY FLOAT,
            AMT_APPLICATION FLOAT,
            AMT_CREDIT FLOAT,
            AMT_DOWN_PAYMENT FLOAT,
            AMT_GOODS_PRICE FLOAT,
            WEEKDAY_APPR_PROCESS_START STRING,
            HOUR_APPR_PROCESS_START INT,
            FLAG_LAST_APPL_PER_CONTRACT STRING,
            NFLAG_LAST_APPL_IN_DAY INT,
            RATE_DOWN_PAYMENT FLOAT,
            RATE_INTEREST_PRIMARY FLOAT,
            RATE_INTEREST_PRIVILEGED FLOAT,
            NAME_CASH_LOAN_PURPOSE STRING,
            NAME_CONTRACT_STATUS STRING,
            DAYS_DECISION INT,
            NAME_PAYMENT_TYPE STRING,
            CODE_REJECT_REASON STRING,
            NAME_TYPE_SUITE STRING,
            NAME_CLIENT_TYPE STRING,
            NAME_GOODS_CATEGORY STRING,
            NAME_PORTFOLIO STRING,
            NAME_PRODUCT_TYPE STRING,
            CHANNEL_TYPE STRING,
            SELLERPLACE_AREA INT,
            NAME_SELLER_INDUSTRY STRING,
            CNT_PAYMENT INT,
            NAME_YIELD_GROUP STRING,
            PRODUCT_COMBINATION STRING,
            DAYS_FIRST_DRAWING INT,
            DAYS_FIRST_DUE INT,
            DAYS_LAST_DUE_1ST_VERSION INT,
            DAYS_LAST_DUE INT,
            DAYS_TERMINATION INT,
            NFLAG_INSURED_ON_APPROVAL INT
        )
        ROW FORMAT DELIMITED
        FIELDS TERMINATED BY ','
        LOCATION 's3://team1ads508finalproject/'
    """,
    'application': """
        CREATE EXTERNAL TABLE IF NOT EXISTS application (
            SK_ID_CURR INT,
            TARGET INT,
            NAME_CONTRACT_TYPE STRING,
            CODE_GENDER STRING,
            FLAG_OWN_CAR STRING,
            FLAG_OWN_REALTY STRING,
            CNT_CHILDREN INT,
            AMT_INCOME_TOTAL FLOAT,
            AMT_CREDIT FLOAT,
            AMT_ANNUITY FLOAT,
            AMT_GOODS_PRICE FLOAT,
            NAME_TYPE_SUITE STRING,
            NAME_INCOME_TYPE STRING,
            NAME_EDUCATION_TYPE STRING,
            NAME_FAMILY_STATUS STRING,
            NAME_HOUSING_TYPE STRING,
            REGION_POPULATION_RELATIVE FLOAT,
            DAYS_BIRTH INT,
            DAYS_EMPLOYED INT,
            DAYS_REGISTRATION INT,
            DAYS_ID_PUBLISH INT,
            OWN_CAR_AGE INT,
            FLAG_MOBIL INT,
            FLAG_EMP_PHONE INT,
            FLAG_WORK_PHONE INT,
            FLAG_CONT_MOBILE INT,
            FLAG_PHONE INT,
            FLAG_EMAIL INT,
            OCCUPATION_TYPE STRING,
            CNT_FAM_MEMBERS FLOAT,
            REGION_RATING_CLIENT INT,
            REGION_RATING_CLIENT_W_CITY INT,
            WEEKDAY_APPR_PROCESS_START STRING,
            HOUR_APPR_PROCESS_START INT,
            REG_REGION_NOT_LIVE_REGION INT,
            REG_REGION_NOT_WORK_REGION INT,
            LIVE_REGION_NOT_WORK_REGION INT,
            REG_CITY_NOT_LIVE_CITY INT,
            REG_CITY_NOT_WORK_CITY INT,
            LIVE_CITY_NOT_WORK_CITY INT,
            ORGANIZATION_TYPE STRING,
            EXT_SOURCE_1 FLOAT,
            EXT_SOURCE_2 FLOAT,
            EXT_SOURCE_3 FLOAT,
            APARTMENTS_AVG FLOAT,
            BASEMENTAREA_AVG FLOAT,
            YEARS_BEGINEXPLUATATION_AVG FLOAT,
            YEARS_BUILD_AVG FLOAT,
            COMMONAREA_AVG FLOAT,
            ELEVATORS_AVG FLOAT,
            ENTRANCES_AVG FLOAT,
            FLOORSMAX_AVG FLOAT,
            FLOORSMIN_AVG FLOAT,
            LANDAREA_AVG FLOAT,
            LIVINGAPARTMENTS_AVG FLOAT,
            LIVINGAREA_AVG FLOAT,
            NONLIVINGAPARTMENTS_AVG FLOAT,
            NONLIVINGAREA_AVG FLOAT,
            APARTMENTS_MODE FLOAT,
            BASEMENTAREA_MODE FLOAT,
            YEARS_BEGINEXPLUATATION_MODE FLOAT,
            YEARS_BUILD_MODE FLOAT,
            COMMONAREA_MODE FLOAT,
            ELEVATORS_MODE FLOAT,
            ENTRANCES_MODE FLOAT,
            FLOORSMAX_MODE FLOAT,
            FLOORSMIN_MODE FLOAT,
            LANDAREA_MODE FLOAT,
            LIVINGAPARTMENTS_MODE FLOAT,
            LIVINGAREA_MODE FLOAT,
            NONLIVINGAPARTMENTS_MODE FLOAT,
            NONLIVINGAREA_MODE FLOAT,
            APARTMENTS_MEDI FLOAT,
            BASEMENTAREA_MEDI FLOAT,
            YEARS_BEGINEXPLUATATION_MEDI FLOAT,
            YEARS_BUILD_MEDI FLOAT,
            COMMONAREA_MEDI FLOAT,
            ELEVATORS_MEDI FLOAT,
            ENTRANCES_MEDI FLOAT,
            FLOORSMAX_MEDI FLOAT,
            FLOORSMIN_MEDI FLOAT,
            LANDAREA_MEDI FLOAT,
            LIVINGAPARTMENTS_MEDI FLOAT,
            LIVINGAREA_MEDI FLOAT,
            NONLIVINGAPARTMENTS_MEDI FLOAT,
            NONLIVINGAREA_MEDI FLOAT,
            FONDKAPREMONT_MODE STRING,
            HOUSETYPE_MODE STRING,
            TOTALAREA_MODE FLOAT,
            WALLSMATERIAL_MODE STRING,
            EMERGENCYSTATE_MODE STRING,
            OBS_30_CNT_SOCIAL_CIRCLE FLOAT,
            DEF_30_CNT_SOCIAL_CIRCLE FLOAT,
            OBS_60_CNT_SOCIAL_CIRCLE FLOAT,
            DEF_60_CNT_SOCIAL_CIRCLE FLOAT,
            DAYS_LAST_PHONE_CHANGE INT,
            FLAG_DOCUMENT_2 INT,
            FLAG_DOCUMENT_3 INT,
            FLAG_DOCUMENT_4 INT,
            FLAG_DOCUMENT_5 INT,
            FLAG_DOCUMENT_6 INT,
            FLAG_DOCUMENT_7 INT,
            FLAG_DOCUMENT_8 INT,
            FLAG_DOCUMENT_9 INT,
            FLAG_DOCUMENT_10 INT,
            FLAG_DOCUMENT_11 INT,
            FLAG_DOCUMENT_12 INT,
            FLAG_DOCUMENT_13 INT,
            FLAG_DOCUMENT_14 INT,
            FLAG_DOCUMENT_15 INT,
            FLAG_DOCUMENT_16 INT,
            FLAG_DOCUMENT_17 INT,
            FLAG_DOCUMENT_18 INT,
            FLAG_DOCUMENT_19 INT,
            FLAG_DOCUMENT_20 INT,
            FLAG_DOCUMENT_21 INT,
            AMT_REQ_CREDIT_BUREAU_HOUR FLOAT,
            AMT_REQ_CREDIT_BUREAU_DAY FLOAT,
            AMT_REQ_CREDIT_BUREAU_WEEK FLOAT,
            AMT_REQ_CREDIT_BUREAU_MON FLOAT,
            AMT_REQ_CREDIT_BUREAU_QRT FLOAT,
            AMT_REQ_CREDIT_BUREAU_YEAR FLOAT
        )
        ROW FORMAT DELIMITED
        FIELDS TERMINATED BY ','
        LOCATION 's3://team1ads508finalproject/'
    """
}

# Create tables in Athena
for table_name, schema in schemas.items():
    print(f"Creating table: Credit")
    cursor = conn.cursor()
    cursor.execute(schema)

# Show tables 
query = "SHOW TABLES"
df_show = pd.read_sql(query, conn)
df_show.head(5)

Creating table: Credit
Creating table: Credit
Creating table: Credit


/tmp/ipykernel_1926/1531778931.py:210: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(query, conn)


,tab_name
0,application
1,bureau
2,previous_application


In [164]:
# Define the SQL query for joining the tables
query = """
SELECT *
FROM application a
INNER JOIN bureau b ON a.SK_ID_CURR = b.SK_ID_CURR
INNER JOIN previous_application p ON a.SK_ID_CURR = p.SK_ID_CURR;
"""

# Execute the SQL query
df = pd.read_sql(query, conn)

# Display the result
print(df)

/tmp/ipykernel_1926/992070815.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


DatabaseError: Execution failed on sql: 
SELECT *
FROM application a
INNER JOIN bureau b ON a.SK_ID_CURR = b.SK_ID_CURR
INNER JOIN previous_application p ON a.SK_ID_CURR = p.SK_ID_CURR;

com.amazonaws.services.s3.model.AmazonS3Exception: Access Denied (Service: Amazon S3; Status Code: 403; Error Code: AccessDenied; Request ID: JFZRW79A7QQ6AN7X; S3 Extended Request ID: HcE/3iyK2tnu+6bYMvr3h65eG0IUGRGcBZ1OjIn3e0oNwqqXBgfwCruSXKBeBL+VItwzOppo+IdjbF+kR5thrZS1tTdi7wPk; Proxy: null), S3 Extended Request ID: HcE/3iyK2tnu+6bYMvr3h65eG0IUGRGcBZ1OjIn3e0oNwqqXBgfwCruSXKBeBL+VItwzOppo+IdjbF+kR5thrZS1tTdi7wPk (Bucket: your-s3-bucket, Key: path/to/previous_application/)
unable to rollback

In [ ]:
#app_df = pd.read_csv('s3://team1ads508finalproject/application_train.csv')

In [ ]:
#app_df

In [ ]:
#prev_app_df = pd.read_csv('s3://team1ads508finalproject/previous_application.csv')

In [ ]:
#prev_app_df

In [ ]:
#bureau_df = pd.read_csv('s3://team1ads508finalproject/bureau.csv')

In [ ]:
#bureau_df